# CALFEWS: California Food-Energy-Water System simulation model
Intro to model, direct to future paper location
## Software setup
* Instructions for downloading from GitHub
* Requires Python 3 (I use Python 3.6.9)  

### Create virtual environment

In [1]:
#! python3 -m venv .venv_calfews
#! source .venv_calfews/bin/activate

### Download Python modules (pip)

In [2]:
# ! pip3 install -r requirements.txt
# ! ipython3 kernel install --user --name=jupyter_calfews

### Download Python modules (conda)

In [3]:
#! conda install --file requirements.txt

### Check dependencies met

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import calfews_src
from calfews_src import *
from datetime import datetime
import os
import shutil
import sys
from configobj import ConfigObj
import json
import h5py
from distutils.util import strtobool
import warnings
warnings.filterwarnings('ignore')

## Model setup
### Parameter file: `calfews_src/data/input/runtime_params.ini`
Intro to file, how to use it

In [5]:
! sed -i 's+\"simulation\"+\"validation\"+' runtime_params.ini

#! cat calfews_src/data/input/runtime_params.ini

In [6]:
startTime = datetime.now()

# get runtime params from config file
config = ConfigObj('runtime_params.ini')
parallel_mode = bool(strtobool(config['parallel_mode']))
parallel_mode, type(parallel_mode)

(False, bool)

In [7]:
model_mode = config['model_mode']
model_mode, type(model_mode)

('validation', str)

In [8]:
short_test = int(config['short_test'])
short_test, type(short_test)

(-1, int)

In [9]:
print_log = bool(strtobool(config['print_log']))
print_log, type(print_log)

(False, bool)

In [10]:
seed = int(config['seed'])
seed, type(seed)

(1001, int)

In [11]:
scenario_name = config['scenario_name'] #scenarios provide information on infrastructural plans
scenario_name, type(scenario_name)

('baseline_wy2017', str)

In [12]:
flow_input_type = config['flow_input_type']
flow_input_type, type(flow_input_type)

('observations', str)

In [13]:
flow_input_source = config['flow_input_source']
flow_input_source, type(flow_input_source)

('CDEC', str)

In [14]:
total_sensitivity_factors = int(config['total_sensitivity_factors'])
total_sensitivity_factors, type(total_sensitivity_factors)

(1, int)

In [15]:
sensitivity_sample_file = config['sensitivity_sample_file']
sensitivity_sample_file, type(sensitivity_sample_file)

('calfews_src/data/input/SALib-sample.txt', str)

In [16]:
output_list = config['output_list']
output_list, type(output_list)

('calfews_src/data/input/output_list.json', str)

In [17]:
output_directory = config['output_directory']
output_directory, type(output_directory)

('results', str)

In [18]:
clean_output = bool(strtobool(config['clean_output']))
clean_output, type(clean_output)

(True, bool)

In [19]:
save_full = bool(strtobool(config['save_full']))
save_full, type(save_full)

(True, bool)

### Inflow source parameter file: `calfews_src/data/input/base_inflows.json`
Intro to file, how to use it

In [20]:
#! cat calfews_src/data/input/base_inflows.json

### Infrastructure scenario file: e.g., `calfews_src/scenarios/scenarios_main.json`
Intro to file, how to use it

In [21]:
#! cat calfews_src/scenarios/scenarios_main.json

In [22]:
#! cat calfews_src/scenarios/FKC_properties__capacity_wy2017.json

### Data output parameter file: `calfews_src/data/input/output_list.json`
Intro to file, how to use it

In [23]:
#! cat calfews_src/data/input/output_list.json